In [4]:
#pip install geemap

In [5]:
import ee
import geemap

In [6]:
Map_S5P = geemap.Map()
Map_S5P

Enter verification code: 4/1AX4XfWifDT-wootinFQrpi_5ZIeRwO9jwhKVBQwo-pNtR_l0X26q83ix6Jk

Successfully saved authorization token.


Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [7]:
point_S5P = ee.Geometry.Rectangle([-122.8003, 37.4831, -122.8036, 37.8288])

image_S5P = ee.ImageCollection("COPERNICUS/S5P/NRTI/L3_NO2") \
    .filterBounds(point_S5P) \
    .filterDate('2018-07-11', '2019-06-30') \
    .sort('ALGORITHM_VERSION') \
    .first() \
    .select('NO2_column_number_density','tropospheric_NO2_column_number_density','stratospheric_NO2_column_number_density')
    #.select('NO2_column_number_density','tropospheric_NO2_column_number_density','stratospheric_NO2_column_number_density')

vis_params = {
    'min': 0,
    'max': 0.0002,
    'bands': ['NO2_column_number_density','tropospheric_NO2_column_number_density','stratospheric_NO2_column_number_density']#,
    #'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}
Map_S5P.centerObject(point_S5P, 8)
Map_S5P.addLayer(image_S5P, vis_params, "Sentinel-5P TROPIMI NO2")


In [8]:
Map_LANDSAT8 = geemap.Map()
Map_LANDSAT8

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [9]:
point_LANDSAT8 = ee.Geometry.Point([-122.4439, 37.7538])

image_LANDSAT8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(point_LANDSAT8) \
    .filterDate('2018-07-01', '2019-06-30') \
    .sort('CLOUD_COVER') \
    .first() \
    .select('B[1-7]')

vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map_S5P.centerObject(point_LANDSAT8, 8)
Map_S5P.addLayer(image_LANDSAT8, vis_params, "Landsat-8")


In [10]:
ground_truth= image_LANDSAT8.addBands(image_S5P)

In [11]:
#tropimi = ee.Image('USGS/NLCD/NLCD2016').select('landcover').clip(image_S5P.geometry())
Map_LANDSAT8.addLayer(image_S5P, {}, 'TROPIMI')
Map_LANDSAT8

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [12]:
# Make the training dataset.
points = ground_truth.sample(**{
    'region': ground_truth.geometry(),
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map_S5P.addLayer(points, {}, 'training', False)

In [13]:
print(points.size().getInfo())

3327


In [14]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-121.68730640159825, 37.86722226931812]}, 'id': '0', 'properties': {'B1': 652, 'B2': 889, 'B3': 1427, 'B4': 2152, 'B5': 3359, 'B6': 3883, 'B7': 2410, 'NO2_column_number_density': 0.00011661837197607383, 'stratospheric_NO2_column_number_density': 5.4645548516418785e-05, 'tropospheric_NO2_column_number_density': 6.197282345965505e-05}}


In [15]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


# This property of the table stores the land cover labels.
label = "NO2_column_number_density","tropospheric_NO2_column_number_density","stratospheric_NO2_column_number_density"
# Overlay the points on the imagery to get training.
training = ground_truth.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

#str1 = ''.join(list1)

In [16]:
#label=str(label).strip('[]')

In [17]:
type(label)

tuple

In [18]:
label[0]

'NO2_column_number_density'

In [19]:
type(bands)

list

In [20]:
# Train a CART classifier with default parameters.
trained = ee.Classifier.smileRandomForest(10).setOutputMode('REGRESSION').train(training, label, bands)

In [21]:
# Classify the image with the same bands used for training.
result = ground_truth.select(bands).classify(trained)

# # Display the clusters with random colors.
Map_LANDSAT8.addLayer(result.randomVisualizer(), {}, 'classfied')
Map_LANDSAT8

EEException: Classifier.train, argument 'classProperty': Invalid type.
Expected type: String.
Actual type: List<String>.
Actual value: [NO2_column_number_density, tropospheric_NO2_column_number_density, stratospheric_NO2_column_number_density]